In [5]:
# from IPython.display import display, HTML

# css_style = """
# <style>
# body {
#     background: linear-gradient(to right, #ff7e5f, #feb47b);
# }
# </style>
# # """

# display(HTML(css_style))


In [6]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, widgets, HBox, VBox

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import ipywidgets as widgets
from ipywidgets import interact

df = pd.read_csv('stroke.csv')
def categorize_age(age):
    if age <= 45:
        return '0-45'
    elif 45 < age <= 60:
        return '45-60'
    elif 60 < age <=70:
        return '60-70'
    elif 70 < age <= 80:
        return '70-80'
    else:
        return '80+'

# Apply the function to create a new column 'Age Group'
df['Age Group'] = df['Age'].apply(categorize_age)

In [7]:
df['ICU Checkout Time'] = pd.to_datetime(df['ICU Checkout Time'])
df['ICU Arrival Time'] = pd.to_datetime(df['ICU Arrival Time'])
df['ICU Stay (Days)'] = (df['ICU Checkout Time'] - df['ICU Arrival Time']).dt.total_seconds() / (60 * 60 * 24)


df['Admission Time'] = pd.to_datetime(df['Admission Time'])
df['Discharge Time'] = pd.to_datetime(df['Discharge Time'])
df['HospitalStay'] = (df['Discharge Time'] - df['Admission Time']).dt.total_seconds() / (60 * 60 * 24)



In [8]:

Gender_widget = widgets.Select(options=['All', 'Female', 'Male'], value='All', description='Gender')
Comorbidities_widget = widgets.Checkbox(value=True, description='Comorbidities:')
AnalysisType_widget = widgets.RadioButtons(options=['Hospital Stay', 'ICU Stay'], value='Hospital Stay', description='')

# Arranging widgets
ui = VBox([HBox([Gender_widget, AnalysisType_widget]), Comorbidities_widget])

def plot_demographic(Gender, Comorbidities, AnalysisType):
    if Gender == 'Female':
        filtered_df = df[df['Gender'] == 'F']
    elif Gender == 'Male':
        filtered_df = df[df['Gender'] == 'M']
    else:
        filtered_df = df

    df_com = filtered_df[filtered_df['Comorbidities'] == True]
    df_com_not = filtered_df[filtered_df['Comorbidities'] == False]

    if Comorbidities:
        filtered_df = df_com
    else:
        filtered_df = df_com_not

    if AnalysisType == 'Hospital Stay':
        average_stay_by_age_group = filtered_df.groupby('Age Group')['HospitalStay'].mean().reset_index()
        y_label = 'Average Hospital Stay (Days)'
        title = 'Average Time Spent in a Hospital (days)'
        y = 'HospitalStay'
    else: # ICU Analysis
        average_stay_by_age_group = filtered_df.groupby('Age Group')['ICU Stay (Days)'].mean().reset_index()
        y_label = 'Average ICU Stay (Days)'
        title = 'Average ICU Stay'
        y = 'ICU Stay (Days)'

    plt.figure(figsize=(8, 5.5))
    bars = sns.barplot(x='Age Group', y=y, data=average_stay_by_age_group, palette='Paired')
    for bar in bars.patches:
        bars.annotate(format(bar.get_height(), '.1f'), 
                      (bar.get_x() + bar.get_width() / 2, 
                       bar.get_height()), ha='center', va='center',
                       size=10, xytext=(0, 8),
                       textcoords='offset points',
                      bbox=dict(boxstyle="round,pad=0.2", facecolor='grey',alpha=0.2))  # Bounding box


    plt.tight_layout()

    plt.title(title)
    plt.xlabel('Age Group')
    # plt.xticks(rotation=45, fontsize=12)

    plt.ylabel(y_label)
    plt.show()

from ipywidgets import interactive_output
out = interactive_output(plot_demographic, {'Gender': Gender_widget, 'Comorbidities': Comorbidities_widget, 'AnalysisType': AnalysisType_widget})

display(ui, out)

Output()

In [9]:
from IPython.display import HTML, display
import ipywidgets as widgets

# Create an HTML widget with the desired statement.
# Here I'm using <blockquote> for semantic emphasis and <strong> to highlight the age specification.
txt = widgets.HTML(
    value="""<blockquote>According to Canadian Best Stroke Practices, referral to a healthcare professional 
    with stroke expertise should be considered for patients with a suspected uncommon cause of stroke, 
    including for <strong>young patients with stroke (e.g., &lt;45 years of age)</strong>.</blockquote>""",
    layout=widgets.Layout(margin='10px', padding='10px', border='2px solid red', width='auto')
)

# Display the widget in a layout that stands out.
display(txt)


HTML(value='<blockquote>According to Canadian Best Stroke Practices, referral to a healthcare professional \n …